In [1]:
!pip install requests
!pip install Pillow
!pip install tqdm
!pip install Cmake
!pip install dlib

!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!git clone https://github.com/hardik1115/Epoching_StyleGan2_Setup.git

fatal: destination path 'Epoching_StyleGan2_Setup' already exists and is not an empty directory.


In [3]:
import shutil
from pathlib import Path
import os
repo_root = Path('Epoching_StyleGan2_Setup/')

# Pull contents out of the repo, into our current directory.
if os.path.exists(repo_root):
  for content in repo_root.iterdir():
    shutil.move(str(content), '.')

  shutil.rmtree(repo_root)

Error: Destination path './.git' already exists

In [ ]:
import sys
sys.path.append('stylegan2/')

from stylegan2 import pretrained_networks
from stylegan2 import dnnlib
from stylegan2.dnnlib import tflib
!pip install tensorflow
from pathlib import Path
from PIL import Image
import pickle
import numpy as np

import ipywidgets as widgets
from tqdm import tqdm


In [ ]:
#T-shirts
!gdown --id 1-jmlEpNCMCwbFCBKbF2Ag117tzurdwVx
!mv network-snapshot-010579.pkl generator_model.pkl

In [ ]:
# HandBags
!gdown --id 165bONdiy6W8tt0aHiWarS8cCjTsWSRYO
!mv network-snapshot-020475.pkl generator_model.pkl

In [ ]:
# WebApp
!gdown --id 11yg3GwNm7aueoaJEykvW36oTq9R8mwU3
!unzip /content/ProductDesign.zip

In [ ]:
fps = 16
results_size = 256

In [ ]:
# Code to load the StyleGAN2 Model
def load_model():
    _G, _D, Gs = pretrained_networks.load_networks('/content/generator_model.pkl')
    
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    
    return Gs, noise_vars, Gs_kwargs

# Generate images given a random seed (Integer)
def generate_image_random(rand_seed):
    rnd = np.random.RandomState(rand_seed)
    z = rnd.randn(1, *Gs.input_shape[1:])
    tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars})
    images = Gs.run(z, None, **Gs_kwargs)
    return images, z

# Generate images given a latent code ( vector of size [1, 512] )
def generate_image_from_z(z):
    images = Gs.run(z, None, **Gs_kwargs)
    return images
  

Gs, noise_vars, Gs_kwargs = load_model()

In [ ]:
import pandas as pd
from random import randrange

image_map_data = []
image_map_dimensions = 10
# image_array=[1,100,200,300,400,500,600,700,800]
for i in range(image_map_dimensions * image_map_dimensions):
  image, latent_code = generate_image_random(randrange(1000))
  # print(Image.fromarray(image[0]).resize((results_size, results_size)).save('pos_' + str(i)+'.png'))
  image_map_data.append(
      {
        'id': 'pos_' + str(i),
        'image': image,
        'latent_vector':  latent_code
      })

image_map_data = pd.DataFrame(image_map_data)
Image.fromarray(image[0]).resize((results_size, results_size))

In [ ]:
import matplotlib.pyplot as plt

def plot_image_map(image_map_data, image_map_dimensions):
  fig = plt.figure(figsize=(image_map_dimensions * 4.6, image_map_dimensions * 4.6))
  ax = [plt.subplot(image_map_dimensions, image_map_dimensions, i+1) for i in range(image_map_dimensions * image_map_dimensions)]

  i = 0
  for a in ax:
      a.imshow(Image.fromarray(image_map_data['image'][i][0]).resize((results_size, results_size)), interpolation='nearest')
      a.set_xticklabels([])
      a.axes.get_xaxis().set_visible(False)
      a.set_yticklabels([])
      a.axes.get_yaxis().set_visible(False)

      i = i+1
      #a.set_aspect('equal')

  fig.subplots_adjust(wspace=0, hspace=0)
  # plt.show()
  plt.subplots_adjust(wspace=0, hspace=0)
  plt.savefig('/content/ProductDesign/img/handbags-fakes4.png',bbox_inches = 'tight',pad_inches=0)

plot_image_map(image_map_data, image_map_dimensions)

In [ ]:
def get_interpolated_vector(latent_vectors, proportions):
  return sum([vector*proportion for vector, proportion in zip(latent_vectors, proportions)])

def linear_interpolate(code1, code2, alpha):
    return code1 * alpha + code2 * (1 - alpha)

In [ ]:
output_gifs_path = Path('output_gifs')
# Make Output Gifs folder if it doesn't exist.
if not output_gifs_path.exists():
    output_gifs_path.mkdir()

In [ ]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def make_latent_interp_animation(code1, code2, img1, img2, num_interps = 10):
    
    step_size = 1.0/num_interps
    
    all_imgs = []
    
    amounts = np.arange(0, 1, step_size)
    
    for alpha in tqdm(amounts):
        interpolated_latent_code = linear_interpolate(code1, code2, alpha)
        images = generate_image_from_z(interpolated_latent_code)
        interp_latent_image = Image.fromarray(images[0]).resize((results_size, results_size))
        frame = get_concat_h(img1, interp_latent_image)
        frame = get_concat_h(frame, img2)
        #frame = interp_latent_image
        all_imgs.append(frame)

    save_name = output_gifs_path/'latent_space_traversal.gif'
    all_imgs[0].save(save_name, save_all=True, append_images=all_imgs[1:], duration=1000/fps, loop=0)

In [ ]:
vector_list_1 = [image_map_data['latent_vector'][randrange(image_map_dimensions*image_map_dimensions)] for i in range(4)]
proportion_list_1 = [0.15, 0.15, 0.35, 0.35]
latent_code1 = get_interpolated_vector(vector_list_1, proportion_list_1)

image1 = Image.fromarray(generate_image_from_z(latent_code1)[0]).resize((results_size, results_size))
#image1

In [ ]:
vector_list_2 = [image_map_data['latent_vector'][randrange(image_map_dimensions*image_map_dimensions)] for i in range(4)]
proportion_list_2 = [0.25, 0.25, 0.25, 0.25]
latent_code2 = get_interpolated_vector(vector_list_2, proportion_list_2)

image2 = Image.fromarray(generate_image_from_z(latent_code2)[0]).resize((results_size, results_size))
#image2

In [ ]:
make_latent_interp_animation(latent_code2, latent_code1, image1, image2, num_interps=64)

In [ ]:
from IPython.display import Image as ipyimg
ipyimg(open('/content/output_gifs/latent_space_traversal.gif','rb').read())

#API Endpoint

In [ ]:
!pip install flask-ngrok
!pip install flask==0.12.2

In [ ]:
!pip install flask_cors
!pip install Flask-JSGlue

In [ ]:
import pandas
!wget https://raw.githubusercontent.com/Ayush9719/Test/main/100-handbags.pkl

# image_map_data = pd.read_csv("2500_handbag_images_df.csv")

import pandas as pd
df = pd.read_pickle("100-handbags.pkl")
# print(image_map_data.head())
# from ast import literal_eval
# df['latent_vector'] = pd.eval(df['latent_vector'])
# df['latent_vector']=df['latent_vector'].apply(lambda x: literal_eval(x))
print(df['latent_vector'][0][0])
type(df['latent_vector'][0][0])


In [ ]:
import pandas as pd
from random import randrange


image_map_dimensions = 3

def make_image_map(image_map_dimensions = 3):
  image_map_data = []
  for i in range(image_map_dimensions * image_map_dimensions):
    image, latent_code = generate_image_random(randrange(1000000))
    image_map_data.append(
        {
          'id': 'pos_' + str(i),
          'image': image,
          'latent_vector':  latent_code
        })

  image_map_data = pd.DataFrame(image_map_data)
  
  return image_map_data

In [ ]:
from sklearn.decomposition import KernelPCA
import random


image_map_data = make_image_map(100)

latent_vectors = [x[0] for x in image_map_data['latent_vector']]





In [ ]:

transformer = KernelPCA(n_components=10, kernel='sigmoid', fit_inverse_transform = True)
transformer = transformer.fit(latent_vectors)



# inv_transformed_vectors = transformer.inverse_transform(reduced_vectors)


In [ ]:

reduced_vectors = transformer.transform(latent_vectors)
noise = np.std(reduced_vectors, axis = 0)

In [ ]:
import time
def make_image_map_kernelPCA(anchor_image_code, image_map_dimensions = 3):
  similarimages_map_data = []
  embedding = transformer.transform(anchor_image_code)
  k = 0
  for i in range(image_map_dimensions):
    for j in range(image_map_dimensions):
      
      start_time = time.time()

      embedding = transformer.transform(anchor_image_code)[0]
      embedding = [[random.uniform(embedding[x] - noise[x]/2, embedding[x] + noise[x]/2) for x in range(10)]]
      

      latent_code = transformer.inverse_transform(embedding)
      image = generate_image_from_z(latent_code)

      #img_sv = Image.fromarray(image[0])
      #img_sv.save('images_separate/'+str(k)+'.jpg', 'JPEG')

      time_taken = time.time() - start_time

      #print("i, j :" , [i, j])
      #print("Reduced Anchor + Noise: ", embedding)
      #print("Time Taken: ", time_taken)
      #print("\n")

      #anchor_image_vector = latent_code
      
      similarimages_map_data.append(
          {
            'id': 'pos_' + str(k),
            'image': image,
            'latent_vector':  latent_code
          })
      k = k+1  
  similarimages_map_data = pd.DataFrame(similarimages_map_data)
  
  return similarimages_map_data


anchor_image_code = np.array([latent_vectors[1140]])
kernelPCA_map_data = make_image_map_kernelPCA(anchor_image_code, image_map_dimensions = 5)

plot_image_map(kernelPCA_map_data, 5)

In [ ]:
img ="/content/ProductDesign/img/handbags-fakes4.png"
im = Image.open(img)

buffer = io.BytesIO()
im.save(buffer, format="PNG")

In [ ]:
from flask import request, render_template,current_app
from flask_cors import CORS
import base64
# flask_ngrok_example.py
from flask import Flask
from flask import Response
from flask_ngrok import run_with_ngrok
import io
from flask import make_response
from flask import jsonify

app = Flask(__name__, template_folder='/content/ProductDesign/',static_folder='/content/ProductDesign/',static_url_path='' )
# app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False
run_with_ngrok(app)  # Start ngrok when app is run
CORS(app)
cors = CORS(app, resource={
    r"/*":{
        "origins":"*"
    }
})

global latent_vector 
@app.route("/image", methods=['POST'])
def send_interpolated_image1():
  print('Hello!')
  response = request.get_json() #New [['pos_1', 0.2], ['pos_2', 0.2], ['pos_3', 0.3], ['pos_4', 0.3]]  ## Old{'pos_1': 0.2,'pos_2': 0.2,'pos_3': 0.3,'pos_4': 0.3}
  print(response)
  # latent_vector_list = [image_map_data.loc[image_map_data['id'] == x, 'latent_vector'].iloc[0] for x in response.keys()]

  
  latent_vector_list = [df.loc[df['id'] == x[0], 'latent_vector'].iloc[0] for x in response]
  # latent_vector_list = [x[1] for x in response]
  # print(latent_vector_list)
  proportion_list = [x[1] for x in response]

  # print(proportion_list)
  # print(latent_vector_list[0])
  global latent_vector
  latent_vector = get_interpolated_vector(latent_vector_list, proportion_list)
  
  interpolated_image = Image.fromarray(generate_image_from_z(latent_vector)[0]).resize((results_size, results_size))
  
  buffer1 = io.BytesIO()
  interpolated_image.save(buffer1, format="PNG")

  
  # images_array = [buffer1,buffer2]
  
  return Response(buffer1.getvalue(), mimetype='image/png')


@app.route("/image-map", methods=['POST'])
def send_something():
  global latent_vector
  # latent_vector = global latent_vector
  anchor_image_code = np.array(latent_vector)
  kernelPCA_map_data = make_image_map_kernelPCA(anchor_image_code, image_map_dimensions = 5)
  plot_image_map(kernelPCA_map_data, 5)
  
  new_im = Image.open("/content/ProductDesign/img/handbags-fakes4.png")
  buffer2 = io.BytesIO()
  new_im.save(buffer2, format="PNG")
  return Response(buffer2.getvalue(), mimetype='image/png')


@app.route("/", methods=["GET"])
def display_ui1():
  path = os.path.join(current_app.root_path, 'myDailyReports')
  print(path)
  print("In UI")
  return render_template('index.html')
    

if __name__ == '__main__':
     app.run()  # If address is in use, may need to terminate other sessions:
                # Runtime > Manage Sessions > Terminate Other Sessions

In [ ]:
response = [['pos_1', 0.2], ['pos_1', 0.3], ['pos_3', 0.3], ['pos_4', 0.3]]  ## Old{'pos_1': 0.2,'pos_2': 0.2,'pos_3': 0.3,'pos_4': 0.3}
print(response)
# latent_vector_list = [image_map_data.loc[image_map_data['id'] == x, 'latent_vector'].iloc[0] for x in response.keys()]
latent_vector_list = [image_map_data.loc[image_map_data['id'] == x[0], 'latent_vector'].iloc[0] for x in response]
# latent_vector_list = [x[1] for x in response]
print(len(latent_vector_list))
proportion_list = [x[1] for x in response]
print(proportion_list)